# 셀레니움으로 자료 가져오기
* chrome 웹드라이버설치 [설치사이트]( https://chromedriver.chromium.org/downloads)
* 셀레니움참고 [참고사이트](https://galid1.tistory.com/478)

In [4]:
# #선행설치
# !pip install selenium

# !conda install -c conda-forge poppler #pdftotext 선행설치
# !pip install pdftotext

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

import time
import pdftotext # pdf 읽는용도
import pikepdf #lock걸려있는 pdf해제
import fitz # PyMuPDF

import pandas as pd
import numpy as np

import os
import re

# 웹페이지에서 제목을 바로 가져오기

In [6]:
#전체저장 위치 
all_df = pd.DataFrame()

def get_sections():
    global all_df, ho_index
    semi_title_dic = dict()
    ho_index = ''
    try :
        for j in range(2,30):
            section = driver.find_element_by_xpath('//*[@id="container_suite"]/div[4]/div/div/ul/li[{num}]'.format(num =j))
            title = section.text.split('\n')[0]
            category = section.text.split('\n')[1].split(' ')[3]
            try:
                if (len(semi_title_dic[category])>0)&(type(semi_title_dic[category]) != list):
                    values = [semi_title_dic[category]]
                    values.append(title)
                    semi_title_dic[category] = values
                else : 
                    values = semi_title_dic[category]
                    values.append(title)
                    semi_title_dic[category] = values
            except Exception as ex:
                semi_title_dic[category] = title

    except Exception as ex:
        ho_index = section.text.split('\n')[1].split(" ")[1]
        semi_title_dic['date'] = section.text.split('\n')[1].split(" ")[5]
        pass
    # to dataFrame
    for key in semi_title_dic.keys():
        semi_title_dic[key] = str(semi_title_dic[key])
    all_df = pd.concat([all_df,pd.DataFrame(semi_title_dic,index=[ho_index])])

### 내용긁어오기 1개

In [157]:
#전체 진행 (오류나면서 끝나는것이 맞다)
os.chdir('D:/Python/SDI 셀레니움/')
options = Options()
# chrome을 전체화면으로 넓히는 옵션입니다.
options.add_argument('--start-maximized') 

driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#사이트이동
driver.get('http://www.sdi.re.kr')
time.sleep(2)

#세계도시동향 클릭
donghyang = driver.find_element_by_xpath('//*[@id="block-system-main-menu"]/div/ul/li[2]/a')
donghyang.click()
time.sleep(2)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [158]:
#저장위치
final_dict = dict()
final_dict['ho'] = []
final_dict['dates'] = []
final_dict['category'] = []
final_dict['city'] = []
final_dict['title'] = []
final_dict['contents'] = []

In [178]:
one_title = driver.find_element_by_xpath('//*[@id="container_suite"]/div[4]/div/div/ul/li[{num}]/div[1]/h2/a'.format(num =4))
one_title.click()

In [179]:
article = driver.find_element_by_xpath('//*[@id="center"]/article')

In [180]:
#facebook, Twitter의 위치가 계속 변동되어 지우고 위치지정
article = article.text.split('\n')

article.remove('Facebook')
article.remove('Twitter')

In [181]:
#추출
title = article[0]
dates = article[2]
ho = article[6]
category = article[8]
contents = article[9:-1]

city = title.split(" ")[-1]
if ')' in city:
    city = city.replace('(',"").replace(')',"")
else :
    city = np.nan

In [182]:
#저장 
final_dict.get('dates',0).append(dates)
final_dict.get('ho',0).append(ho)
final_dict.get('category',0).append(category)
final_dict.get('city',0).append(city)
final_dict.get('title',0).append(title)
final_dict.get('contents',0).append(contents)

In [183]:
driver.back()

In [189]:
pd.DataFrame(final_dict)

,ho,dates,category,city,title,contents
0,제475호,2020.04.17,헤드라인뉴스,홍콩,‘코로나19’로 피해가 큰 소상공인에게 보조금 지급 (홍콩),[홍콩특별행정구는 ‘코로나19’ 발생으로 타격을 받은 음식점 등 9개 업종의 소상공...
1,제475호,2020.04.17,문화/관광,교토市,市 공식 관광사이트에 ‘채팅봇’ 도입 (일본 교토市),[일본 교토市는 시간이나 장소에 구애받지 않고 관광객의 의문이나 문의사항을 해결함으...
2,제475호,2020.04.17,도시계획/주택,자카르타州,중앙정부와 협력해 대중교통 지향형 개발(TOD) 추진 (인도네시아 자카르타州),[인도네시아 자카르타州는 날로 악화되는 교통혼잡･체증을 해결하고 지속가능한 도시를 ...


In [202]:
try:
    for j in range(2,30):
        one_title = driver.find_element_by_xpath('//*[@id="container_suite"]/div[4]/div/div/ul/li[{num}]/div[1]/h2/a'.format(num =j))
        one_title.click()

        article_xpath = driver.find_element_by_xpath('//*[@id="center"]/article')

        #facebook, Twitter의 위치가 계속 변동되어 지우고 위치지정
        article = article_xpath.text.split('\n')
        article.remove('Facebook')
        article.remove('Twitter')


        #추출
        title = article[0]
        dates = article[2]
        ho = article[6]
        category = article[8]
        contents = article[9:-1]

        city = title.split(" ")[-1]
        if ')' in city:
            city = city.replace('(',"").replace(')',"")
        else :
            city = np.nan

        #저장 
        final_dict.get('dates',0).append(dates)
        final_dict.get('ho',0).append(ho)
        final_dict.get('category',0).append(category)
        final_dict.get('city',0).append(city)
        final_dict.get('title',0).append(title)
        final_dict.get('contents',0).append(contents)

        #뒤로가기
        time.sleep(0.5)
        driver.back()
except :
    pass

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[4]/div/div/ul/li[10]/div[1]/h2/a"}
  (Session info: chrome=81.0.4044.129)


In [197]:
article_xpath.text.split('\n')

['서울시 정책개발과 도시 관련연구에 참고할 가치가 있는 해외 주요도시의 정책사례와 동향을 소개하는 정기간행물입니다.',
 '세계 각지에서 활동 중인 해외통신원으로부터 시의성 있는 사례와 정확한 현지 정보를 전해드리고 있습니다.',
 '7462개의 데이터가 있습니다.',
 '제475호',
 '제474호',
 '제473호',
 '제472호',
 '제471호',
 '제470호',
 '제469호',
 '제468호',
 '제467호',
 '세계도시동향 제475호',
 '권호 제475호 등록일 2020.04.17 조회수 51',
 '원문',
 '‘코로나19’로 피해가 큰 소상공인에게 보조금 지급 (홍콩)',
 '권호 제475호 분야 헤드라인뉴스 등록일 2020.04.17 조회수 157',
 '市 공식 관광사이트에 ‘채팅봇’ 도입 (일본 교토市)',
 '권호 제475호 분야 문화/관광 등록일 2020.04.17 조회수 50',
 '모임･외출 금지 등 ‘코로나19’ 확산 방지대책 강화 (이탈리아)',
 '권호 제475호 분야 사회/복지 등록일 2020.04.17 조회수 143',
 '건물 신축 시 ‘유아놀이방’ 설립 의무화 (인도 델리NCT)',
 '권호 제475호 분야 사회/복지 등록일 2020.04.17 조회수 44',
 '대기오염이 심한 도로에서 노후 디젤자동차 통행 금지 (스웨덴 스톡홀름市)',
 '권호 제475호 분야 환경/안전 등록일 2020.04.17 조회수 20',
 '중앙정부와 협력해 대중교통 지향형 개발(TOD) 추진 (인도네시아 자카르타州)',
 '권호 제475호 분야 도시계획/주택 등록일 2020.04.17 조회수 81',
 '‘임대주택 분쟁방지 가이드라인’ 다국어 서비스 시행 (일본 도쿄都)',
 '권호 제475호 분야 도시계획/주택 등록일 2020.04.17 조회수 22',
 '도시경관을 위해 옥외광고 규제 강화 및 광고세 신설 (네덜란드 암스테르담市)',
 '권호 제475호 분야 도시계획/주택 등록일 2020.04.17 조회수 43',
 '1',

### 함수화

In [246]:
#저장위치
final_dict = dict()
final_dict['ho'] = []
final_dict['dates'] = []
final_dict['category'] = []
final_dict['city'] = []
final_dict['title'] = []
final_dict['contents'] = []

def get_contents():
    try :
        for j in range(2,30):
            one_title = driver.find_element_by_xpath('//*[@id="container_suite"]/div[4]/div/div/ul/li[{num}]/div[1]/h2/a'.format(num =j))
            one_title.click()

            article_xpath = driver.find_element_by_xpath('//*[@id="center"]/article')

            #facebook, Twitter의 위치가 계속 변동되어 지우고 위치지정
            article = article_xpath.text.split('\n')
            article.remove('Facebook')
            article.remove('Twitter')


            #추출
            title = article[0]
            dates = article[2]
            ho = article[6]
            category = article[8]
            contents = article[9:-1]

            city = title.split(" ")[-1]
            if ')' in city:
                city = city.replace('(',"").replace(')',"")
            else :
                city = np.nan

            #저장 
            final_dict.get('dates',0).append(dates)
            final_dict.get('ho',0).append(ho)
            final_dict.get('category',0).append(category)
            final_dict.get('city',0).append(city)
            final_dict.get('title',0).append(title)
            final_dict.get('contents',0).append(contents)

            #뒤로가기
            time.sleep(0.5)
            driver.back()
    except Exception as e:
        pass 


In [217]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [244]:
get_contents()

all 11 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[4]/div/div/ul/li[11]/div[1]/h2/a"}
  (Session info: chrome=81.0.4044.129)



In [247]:
#1페이지 내 다운로드 
for i in range(1,10):
    #처음페이지는 9개인데 이후 10개씩
#     ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
    ho = WebDriverWait(driver, 1).until(EC.presence_of_element_located(
    (By.XPATH, '//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num=i))))
    ho.click()
    time.sleep(0.5)
    get_contents()

all 10 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[4]/div/div/ul/li[10]/div[1]/h2/a"}
  (Session info: chrome=81.0.4044.129)

all 11 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[4]/div/div/ul/li[11]/div[1]/h2/a"}
  (Session info: chrome=81.0.4044.129)

all 10 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[4]/div/div/ul/li[10]/div[1]/h2/a"}
  (Session info: chrome=81.0.4044.129)

all 10 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[4]/div/div/ul/li[10]/div[1]/h2/a"}
  (Session info: chrome=81.0.4044.129)

all 10 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[4]/div/div/ul/li[10]/div[1]/h2/a"}
  (Session info: chrome=81.0.4044.129)

all 8 Message: no such el

In [248]:
for page_num in [2,5,6,7,8,9]:
# for page_num in [2]:
    next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[{page_num}]/a'.format(page_num = page_num))
    next_page.click()
    time.sleep(2)
    try :
        for i in range(1,11):
        #처음페이지는 9개인데 이후 10개씩
            ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
            ho.click()
            time.sleep(2)
            get_contents()
    except:
        pass

all 13 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[4]/div/div/ul/li[13]/div[1]/h2/a"}
  (Session info: chrome=81.0.4044.129)



KeyboardInterrupt: 

# 과정통합

In [5]:
#전체 진행 (오류나면서 끝나는것이 맞다)
os.chdir('D:/Python/SDI 셀레니움/')
options = Options()
# chrome을 전체화면으로 넓히는 옵션입니다.
options.add_argument('--start-maximized') 

driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#사이트이동
driver.get('http://www.sdi.re.kr')
time.sleep(2)

#세계도시동향 클릭
donghyang = driver.find_element_by_xpath('//*[@id="block-system-main-menu"]/div/ul/li[2]/a')
donghyang.click()
time.sleep(2)

#1페이지 내 다운로드 
for i in range(1,10):
    #처음페이지는 9개인데 이후 10개씩
    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
    ho.click()
    time.sleep(2)
    get_sections()
# 2페이지로 넘어가는게 아래의 숫자와 같고 9가 반복됨 (원인은 숫자가 계속 밀리기 때문)
#해당페이지는 2,3,4,5,6,7,
for page_num in [2,5,6,7,8,9]:
# for page_num in [2]:
    next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[{page_num}]/a'.format(page_num = page_num))
    next_page.click()
    time.sleep(2)
    try :
        for i in range(1,11):
        #처음페이지는 9개인데 이후 10개씩
            ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
            ho.click()
            time.sleep(2)
            get_sections()
    except:
        pass
#이후 반복 
while True:
    try:
        #이후반복
        while True:
            next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[9]/a')
            next_page.click()
            driver.implicitly_wait(10)
            for i in range(1,11):
                try:
            #처음페이지는 9개인데 이후 10개씩
                    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
                    ho.click()
                    time.sleep(2)
                    get_sections()
                except:
                    pass
    except:
        #에러난 페이지 내 다운로드 
        ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[2]/ul/li[1]/a')
        ho.click()
        for i in range(1,11):
            try:
                #처음페이지는 9개인데 이후 10개씩
                ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
                ho.click()
                time.sleep(2)
                get_sections()
            except: 
                pass
        pass


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[2]/ul/li[1]/a"}
  (Session info: chrome=80.0.3987.163)


### 과정분할

In [300]:
#1단계
# os.chdir('D:/Python/SDI 샐레니움/')
# options = Options()
# # chrome을 전체화면으로 넓히는 옵션입니다.
# options.add_argument('--start-fullscreen') #이거 케바케로 작동하는 듯함 

# driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#사이트이동
driver.get('http://www.sdi.re.kr')
time.sleep(2)

#세계도시동향 클릭
donghyang = driver.find_element_by_xpath('//*[@id="block-system-main-menu"]/div/ul/li[2]/a')
donghyang.click()
time.sleep(2)

#1페이지 내 다운로드 
for i in range(1,10):
    #처음페이지는 9개인데 이후 10개씩
    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
    ho.click()
    time.sleep(2)
    get_sections()


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [301]:
# 2페이지로 넘어가는게 아래의 숫자와 같고 9가 반복됨 (원인은 숫자가 계속 밀리기 때문)
#해당페이지는 2,3,4,5,6,7,
for page_num in [2,5,6,7,8,9]:
# for page_num in [2]:
    next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[{page_num}]/a'.format(page_num = page_num))
    next_page.click()
    time.sleep(2)
    try :
        for i in range(1,11):
        #처음페이지는 9개인데 이후 10개씩
            ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
            ho.click()
            time.sleep(2)
            get_sections()
    except:
        pass

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [318]:
#중간에 오류날때마다 재실행해주면된다 계속 데이터는 쌓이고 있다
while True:
    try:
        #이후반복
        while True:
            next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[9]/a')
            next_page.click()
            driver.implicitly_wait(10)
            for i in range(1,11):
                try:
            #처음페이지는 9개인데 이후 10개씩
                    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
                    ho.click()
                    time.sleep(2)
                    get_sections()
                except:
                    pass
    except:
        #에러난 페이지 내 다운로드 
        ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[2]/ul/li[1]/a')
        ho.click()
        for i in range(1,11):
            try:
                #처음페이지는 9개인데 이후 10개씩
                ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
                ho.click()
                time.sleep(2)
                get_sections()
            except: 
                pass
        pass



C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[2]/ul/li[1]/a"}
  (Session info: chrome=80.0.3987.163)


In [321]:
all_df.to_csv('중간저장.csv')

### 데이터 수정
* 빈값을 제거하고 하는 과정이기때문에 엑셀작업이 편리하여 엑셀에서 작업하여 가져옴

In [330]:
all_df = pd.read_csv('중간저장.csv')

### 날짜누락추출

In [19]:
data_df = pd.DataFrame(file_list,columns = ['file_name'])

In [31]:
data_df['ho'] = data_df['file_name'].apply(lambda x : x[:4].replace('호',"").replace('(',"").replace('.',''))

In [33]:
data_df['ho'] = data_df['ho'].astype('int')

In [35]:
data_df = data_df.sort_values('ho')

In [41]:
want_df =data_df[(data_df.ho>=136)&(data_df.ho<=272)]

In [43]:
want_df['date'] = want_df['file_name'].apply(lambda x : x.split('(')[1].split(')')[0])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
want_df['date'] = want_df['date'].apply(lambda x : x.replace('년','.').replace('일','').replace('월','.'))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
want_df.to_csv('날짜.csv',encoding = 'cp949')

# pdf read
* 범위에 따른 구분  
    1. 160이전 1차 (목차x)  
    2. 160~? 2차 목차 2페이지 (pattern1)  
    3. 237~300 3차 목차 2페이지 (pattern2)  ->결과 누락 :2건 사유 :299,300 목차없어서 pass  
    4. 301~445	4차 목차 1페이지  
    5. 446~	5차 목차 1페이지(도시명 없음)  

## 3. 237~300 3차 목차 2페이지 (pattern2) 2020-04-08 14:12:58 Done 
결과 누락 :2건 사유 :299,300 목차없어서 pass  

In [9]:
os.chdir('D:/Python/SDI 셀레니움/SDI 리포트')
file_list = os.listdir()

#호수 번호 컬럼생성
num_pattern = re.compile('[0-9]+')
file_df =pd.DataFrame(file_list,columns=['file_name'])
file_df['ho'] = file_df['file_name'].apply(lambda x : re.match(num_pattern,x).group()).astype(int)

set3 = file_df[(file_df['ho']>=237)&(file_df['ho']<=300)].sort_values('ho')

In [30]:
semi_title = ['문화·디자인','산업·경제','건강·복지','도시환경','문화·디자인','복지','일자리 창출']
all_df = pd.DataFrame()
for i in range(0,len(set3)):
    #pdf read
    file_name = set3['file_name'].values[i]
    file = open(file_name, 'rb')
    fileReader = pdftotext.PDF(file)
    page_sentences = fileReader[1].split('\r\n')
    #pdf 내 목차 분리
    #세계도시동향으로 시작하지 않는 이레귤러 배제 
    values = []
    semi_title_dic = dict()
    if page_sentences[0].split(' ')[0] == '세계도시동향':
        #호수 저장     
        ho = int(re.match(num_pattern,file_name).group())
        semi_title_dic['호'] = ho
        for line in range(1,len(page_sentences)):
            #카테고리면 이름만 보존
            check_category = page_sentences[line].replace(' ',"")
            check_category = check_category.replace('ㆍ','·')
            if check_category in semi_title:
                if line != 1:
                    semi_title_dic[col_name] = str(values)
                    values = []
                col_name = check_category
            else :
                values.append(page_sentences[line])
        semi_title_dic[col_name] = str(values)

    all_df = pd.concat([all_df,pd.DataFrame(semi_title_dic,index=[i])])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [41]:
#저장
all_df = all_df[['호','건강·복지', '도시환경', '문화·디자인', '복지', '산업·경제']]
os.chdir('D:/Python/SDI 샐레니움')
all_df.to_csv('SDI_유형3.csv',index= False)

### 4. 301~445	4차 목차 1페이지 2020-04-09 09:13:16 START
* pdftotext 안됨
* fitz 안됨  -> 초반목차 이미지로 추정
* 이미지로 불러와서 추출 시도 -> 완벽한 글자추출 힘듬 

In [10]:
os.chdir('D:/Python/SDI 셀레니움/SDI 리포트')
file_list = os.listdir()

#호수 번호 컬럼생성
num_pattern = re.compile('[0-9]+')
file_df =pd.DataFrame(file_list,columns=['file_name'])
file_df['ho'] = file_df['file_name'].apply(lambda x : re.match(num_pattern,x).group()).astype(int)

set4 = file_df[(file_df['ho']>=301)&(file_df['ho']<=445)].sort_values('ho')

#### pdftotext

In [28]:
semi_title = ['문화·디자인','산업·경제','건강·복지','도시환경','문화·디자인','복지','일자리 창출','도시교통','도시계획·주택',
             '방재·안전','헤드라인뉴스']
all_df = pd.DataFrame()
for i in range(0,len(set4)):
    #pdf read
    file_name = set4['file_name'].values[i]
    file = open(file_name, 'rb')
    fileReader = pdftotext.PDF(file)
    page_sentences = fileReader[0].split('\r\n')
    print(set4['file_name'].values[i])
    print(page_sentences)
    print("----------------------------------")

301호(2012년12월17일)_0.pdf
['']
----------------------------------
302호_0.pdf
['']
----------------------------------
303호_0.pdf
[' 헤 드 라 인 뉴 스 (행 정 ·재 정 )', '1. 시정의 투명성 제고와 디지털기술 개발 촉진 및 공유를 위해', '   ‘시카고 디지털 허브’ 구축 (미국 시카고市)', '  \x81 미국 시카고市는 市가 보유한 모든 행정데이터, 데이터를 가공하는 디지', '     털기술, 데이터를 소셜 미디어를 통해 제공하는 기술 등을 ‘시카고 디지', '     털 허브’를 통해 일반 시민에게 공개하고 이를 활용해 더욱 다양한 기술', '     을 개발하도록 해 정보의 투명성, 시민의 시정 참여도를 높이고자 함.', '     - 市는 디지털 자원을 공유하는 ‘Chicago Digital’을 구축해 시민이 市가', '       보유한 행정데이터에 접근할 수 있도록 하고 市와 사기업이 함께 개발', '       했던 디지털기술을 공개해 그 기술을 기반으로 새로운 기술을 개발할', '       수 있도록 함.', '          <시민이 市의 모든 정보에 접근할 수 있도록 만들어진', '               Digital.CityOfChicago.org 웹사이트>', '                                               서울연구원 :: 1', '']
----------------------------------
304호_0.pdf
[' 헤 드 라 인 뉴 스 (도 시 교 통 )', '1. 앱을 이용해 대중교통 혼잡도 관련 정보 제공 (미국 시카', '   고市 외)', '  \x81 최근 이스라엘 개발자가 개발한 Moovit 시스템이 세계 여러 도시에서', '    운영되고 있음. Moovit은 일반 앱 유저로부터 버스나 지하철의 혼잡정도', '    에 대한 정보를 모아 다시 유저들에게 정보를 제

394호.pdf
['']
----------------------------------
395호.pdf
['']
----------------------------------
396호.pdf
['']
----------------------------------
397호.pdf
['']
----------------------------------
398호.pdf
['']
----------------------------------
399호.pdf
['']
----------------------------------
400호.pdf
['']
----------------------------------
401호.pdf
['']
----------------------------------
402호.pdf
['']
----------------------------------
403호.pdf
['']
----------------------------------
404호.pdf
['']
----------------------------------
405호.pdf
['']
----------------------------------
406호(2017년 5월 1일).pdf
['']
----------------------------------
407호.pdf
['']
----------------------------------
408호.pdf
['']
----------------------------------
409호(2017년 6월 19일).pdf
['']
----------------------------------
410호.pdf
['']
----------------------------------
411호(2017년 7월 17일).pdf
['']
----------------------------------
412호(2017년 8월 7일).pdf
['']
----------------------------------
413호(2017년 8월 2

441호(20181022).pdf
['            2018. 10. 22 441 호', '            세계도시동향', '헤드라인뉴스    1 온라인 진료위탁 의뢰시스템 주 전역 적용          자카르타', '  문화·관광   3 ‘일상의 축제화’로 지속가능한 예술도시로 승화       토론토', '  산업·경제   6 ‘지속가능, 자급자족, 민주적 도시’ 기치 팹시티 주도  바르셀로나', '          9 로봇 등 첨단기술 활용한 민간사업자 실증사업 지원     오사카', '  사회·복지  11 사회주거지 거주민이 쉽게 이사가게 ‘교환 플랫폼’ 설치  파리', '        13  시설보호 아동·청소년, 가정형 보호로 이전 지원 강화   빅토리아', '        16  의무규정 민간사업자로 확대한 ‘장애인 조례’        도쿄', '  환경·안전 18  ‘런던 녹지화’ 시민·지역커뮤니티에 나무 무료 제공    런던', '  도시교통  20  킥보드·스쿠터 등 전기동력 개인이동수단 자율 대여     보르도', '  정보·기타 23  수요자 기호에 따른 공공정보 제공으로 ‘열린 도시 1위’ 에드먼턴', '']
----------------------------------
442호(20181108)_수정.pdf
['            2018. 11. 5 442 호', '            세계도시동향', '헤드라인뉴스    1 공공건설현장, 대기오염물질 배출 ‘차단’           베를린', '  사회·복지   4 사회적 처방 프로그램으로 시민건강 증진            런던', '          6 ‘자살률 제로 만들기’ 자살방지 종합전략 수립        뉴사우스웨일즈', ' 행재정·교육   9 2022년까지 25억유로 투자해 ‘살기좋은 도시’로 탈바꿈 암스테르담', '        12  숙제 지원 등 초등학생 방과후 사업 보강           오사카', '  환경·안전 14  100% 재생 아스콘으로 고속도

In [27]:
fileReader[0]

'           2018. 12. 17 445 호\r\n           세계도시동향\r\n헤드라인뉴스   1 인구유지·지역활성화로 지속발전 도시 실현       도시마\r\n  산업·경제  4 슬라이드 이동형 등 생활지원로봇 실증실험       가나가와\r\n  환경·안전  6 1회용 비닐봉지 사용 금지 등 플라스틱 쓰레기 규제 자카르타\r\n         9 자동차도로·집앞 보행로 제설규정 ‘손질’       에드먼턴\r\n  도시교통  11 접근·안전·저가성 기반 노인특화 교통전략 수립    뉴사우스웨일즈\r\n도시계획·주택 14 옥외광고물 설치부터 관리까지 규제·지도        오사카\r\n        16 촘촘한 관리의 ‘광고물조례’ 별도 제정해 경관보호  보르도\r\n        18 활용도 낮은 가판대 10곳 새 용도 모색       바르셀로나\r\n'

#### fitz

In [23]:
for i in range(0, len(set4)):
    doc = fitz.open(set4['file_name'].values[i])
    page = doc[0]
    page_sentences = page.getText().split('\n')
    page_sentences_s = pd.Series(page_sentences).apply(lambda x : x.replace('\x81','·'))
    print(set4['file_name'].values[i])
    print(page_sentences_s)
    print("----------------------------------")

301호(2012년12월17일)_0.pdf
0    
dtype: object
----------------------------------
302호_0.pdf
0    
dtype: object
----------------------------------
303호_0.pdf
0                                           서울연구원 :: 1
1                            헤 드 라 인 뉴 스 ( 행 정 · 재 정 )
2                 1. 시정의 투명성 제고와 디지털기술 개발 촉진 및 공유를 위해 
3                            ‘시카고 디지털 허브’ 구축 (미국 시카고市)
4             � 미국 시카고市는 市가 보유한 모든 행정데이터, 데이터를 가공하는 디지
5              털기술, 데이터를 소셜 미디어를 통해 제공하는 기술 등을 ‘시카고 디지
6              털 허브’를 통해 일반 시민에게 공개하고 이를 활용해 더욱 다양한 기술
7               을 개발하도록 해 정보의 투명성, 시민의 시정 참여도를 높이고자 함.
8     - 市는 디지털 자원을 공유하는 ‘Chicago Digital’을 구축해 시민이 市가 
9                보유한 행정데이터에 접근할 수 있도록 하고 市와 사기업이 함께 개발
10               했던 디지털기술을 공개해 그 기술을 기반으로 새로운 기술을 개발할 
11                                            수 있도록 함.
12                    <시민이 市의 모든 정보에 접근할 수 있도록 만들어진   
13                     Digital.CityOfChicago.org 웹사이트>
14                                                    
dtype: object
-----

349호.pdf
0    
dtype: object
----------------------------------
350호.pdf
0    
dtype: object
----------------------------------
351호.pdf
0    
dtype: object
----------------------------------
352호.pdf
0    
dtype: object
----------------------------------
353호.pdf
0    
dtype: object
----------------------------------
354호.pdf
0    
dtype: object
----------------------------------
355호.pdf
0    
dtype: object
----------------------------------
356호.pdf
0    
dtype: object
----------------------------------
357호.pdf
0    
dtype: object
----------------------------------
358호0519.pdf
0    
dtype: object
----------------------------------
359호_0528.pdf
0    
dtype: object
----------------------------------
360호.pdf
0    
dtype: object
----------------------------------
361호(0629).pdf
0    
dtype: object
----------------------------------
362호(2015년 7월 6일)_합본.pdf
0    
dtype: object
----------------------------------
363호(2015년 7월 20일)_합본.pdf
0    
dtype: object
---------------------------

437호(2018년 8월 20일).pdf
0                        2018. 8. 20
1                              437 호
2                             세계도시동향
3                             헤드라인뉴스
4                              문화·관광
5                              산업·경제
6                              사회·복지
7                             행재정·교육
8                              환경·안전
9                               도시교통
10                           도시계획·주택
11                                 1
12                                 4
13                                 7
14                                 9
15                                11
16                                13
17                                14
18                                16
19                                18
20                                20
21                                23
22             ‘주택공급 혁신모델’ 임대형 주택 도입
23         ‘도시 우호적·지속가능한 관광’ 새 전략 수립
24    4차 산업혁명 따른 혁신촉진 등 담아 ‘성장전략’ 손질
25               8월부터 유치원 교육비 완전 무료화
26    특수교육 필요 어

#### another

In [20]:
import pytesseract
import pdf2image

In [24]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [15]:
os.chdir('D:/Python/SDI 셀레니움/연습용')

In [113]:
# Read a pdf file as image pages
# We do not want images to be to big, dpi=200
# All our images should have the same size (depends on dpi), width=1654 and height=2340
pages = pdf2image.convert_from_path('350호.pdf',dpi = 100, first_page=1, last_page =1)

# Save all pages as images
for i in range(len(pages)):
    pages[i].save('a' + str(i) + '.jpg')

# Convert a page to a string (page 2)
content = pt.image_to_string(pages[0], lang='kor')
print(content)

셰계도시동항은
서울연구원 홈페이지
10000.511610를 통해서도
보실 수 있습니다.

13071

서올특별시 서초구
님부순환로 340길 57

02-21497234
.511210
70000.6007050001050006
'412080006000019601179리110

헤드라인뉴스

분화관괄

산업경제

시회복지

완경위전

도시교통

점보기타

| 떼 떼길2

 

2015.15          350호

세계도시동향

'하위중산층 도심주거권 보장" 우선매입지 257곳 선정
'지역의 문화를 세계로' 교토 문예 프로그램 2020 수립

구직자에게 취업면접을 위한 맞춤형 복장 조언
유휴 부동산의 리노베이션으로 도시형 비즈니스 집적 추진

시니어 세일즈 등 4개 '고령자 직업 모델" 선정
관계기관 역할 명기한 '에볼라 대응 매뉴얼' 작성

'테러 등 예측 불가능한 위험 대비" 적응유연한 도시전략 수립
"여러 도시에서 사용할 수 있는 주차관리시스팀 구축을"

스마트폰으로 택시를 부르는 애플리케이션 개발

서울연구원

스 7096 500늬1050146@

크스

피리

교토

파리
기타#슈


In [114]:
# # 이미지반절만들기 (필요없어짐)
# from PIL import Image

# a = Image.open('a0.jpg')

# x = a.size[0]/2
# y = a.size[1]/2

# a = a.crop((x,0,a.size[0],a.size[1]))

# a.save('a0.jpg')

# content = pt.image_to_string('a0.jpg', lang='kor')
# print(content)

### pdftotext작동안함

In [43]:
os.chdir('D:/Python/SDI 셀레니움/SDI 리포트')
file_list = os.listdir()

#호수 번호 컬럼생성
num_pattern = re.compile('[0-9]+')
file_df =pd.DataFrame(file_list,columns=['file_name'])
file_df['ho'] = file_df['file_name'].apply(lambda x : re.match(num_pattern,x).group()).astype(int)

set5 = file_df[(file_df['ho']>=446)].sort_values('ho')

In [167]:
for i in range(10,30):
    #pdf read
    file_name = set5['file_name'].values[i]
    file = open(file_name, 'rb')
    fileReader = pdftotext.PDF(file)
    page_sentences = fileReader[0].split('\r\n')
    print(set5['file_name'].values[i])
    print(page_sentences)
    print("----------------------------------")

456호(190530).pdf
['      5. 20446호', '2019.1.1', '2019.      456호', '           직장인 업무상 정신적 지속발전', '헤드라인뉴스 인구유지·지역활성화로', '헤드라인뉴스                        도시 실현지원금', '                         손해 예방사업에            환경·안전', '                                            도시교통     시민참여로 미래 25년', '                                                     접근·안전·저가성      폐기물 관리대책', '                                                                          교통전략 수립', '                                                                기반 노인특화', '                                                     취약계층 거주지 기후변화 대응능력 향상', '산업·경제', '문화·관광      문화·예술이동형', '           슬라이드     등 생활지원로봇', '                 창조산업   가치 키우는실증실험', '                               4대 비전 제시     도시계획·주택  옥외광고물 설치부터 관리까지 규제·지도', '                                                     플라스틱 일회용품 사용 전면 금지법 채택 ‘눈앞’', '           다양한 프로젝트로 친환경·지속가능한 관광 실현', '환경·안전      1회용 비닐봉지 사용 금지 등 플라스틱 쓰레기 규제              일터 떠날', '                             

465.pdf
['      10. 7 446호', '2019.1.1', '2019.       465호', '헤드라인뉴스        ‘준비된·건강한·연결된’', '헤드라인뉴스 인구유지·지역활성화로          자카르타', '                           지속발전    실현탄력성 전략', '                                도시회복        환경·안전', '                                            도시교통     매립지에 태양광기반', '                                                    접근·안전·저가성  발전소…환경·경제효과', '                                                                 노인특화 교통전략 수립‘양득’', '산업·경제                                       도시계획·주택', '                                                     모니터링 시스템 포함 소음공해 완화대책 수립', '사회·복지         강렬한이동형', '            슬라이드        생활지원로봇', '                      등여성폭력', '                  ‘벽보’로      반대 실증실험', '                                메시지 전달              옥외광고물 설치부터 관리까지 규제·지도', '                                                     전문가 파견·철거비 지원으로 노후건축물 관리', '환경·안전         유휴차량', '            1회용 비닐봉지     금지 등복지시설', '                      사용고령자', '                    이용해     

IndexError: index 29 is out of bounds for axis 0 with size 29

# another

In [160]:
for i in range(0, len(set5)):
    doc = fitz.open(set5['file_name'].values[i])
    page = doc[0]
    page_sentences = page.getText().split('\n')
    page_sentences_s = pd.Series(page_sentences).apply(lambda x : x.replace('\x81','·'))
    print(set5['file_name'].values[i])
    print(page_sentences_s)
    print("----------------------------------")

446호(20190108).pdf
0                   ISSN 2586-5102
1                            06756
2                        서울특별시 서초구
3                    남부순환로 340길 57
4                     02-2149-1234
5                     www.si.re.kr
6           인구유지·지역활성화로 지속발전 도시 실현
7           슬라이드 이동형 등 생활지원로봇 실증실험
8     1회용 비닐봉지 사용 금지 등 플라스틱 쓰레기 규제
9           자동차도로·집앞 보행로 제설규정 ‘손질’
10       접근·안전·저가성 기반 노인특화 교통전략 수립
11           옥외광고물 설치부터 관리까지 규제·지도
12     촘촘한 관리의 ‘광고물조례’ 별도 제정해 경관보호
13          활용도 낮은 가판대 10곳 새 용도 모색
14                          헤드라인뉴스
15                           산업·경제
16                           환경·안전
17                            도시교통
18                         도시계획·주택
19                        2019.1.1
20                           446호 
21                      2019. 1. 7
22                            446호
23                          헤드라인뉴스
24    택시 등 모든 교통네트워크, 정크푸드 광고 ‘퇴출’
25                           문화·관광
26       문학·도시 연결하는 ‘프로젝트 북마크 캐나다’
27                           사회·복지
2

0                           헤드라인뉴스
1     ‘준비된·건강한·연결된’ 자카르타 회복 탄력성 전략
2                            사회·복지
3         강렬한 ‘벽보’로 여성폭력 반대 메시지 전달
4          유휴차량 이용해 고령자 복지시설 이동 지원
5                           행재정·교육
6        스타트업·빅테크 기업과 스마트 도시 실현 노력
7                            환경·안전
8        매립지에 태양광 발전소…환경·경제효과 ‘양득’
9         모니터링 시스템 포함 소음공해 완화대책 수립
10        전문가 파견·철거비 지원으로 노후건축물 관리
11                            도시교통
12        전동스쿠터 공유서비스 확산에 따른 규제 마련
13                  ISSN 2586-5102
14                           06756
15                       서울특별시 서초구
16                   남부순환로 340길 57
17                    02-2149-1234
18                    www.si.re.kr
19          인구유지·지역활성화로 지속발전 도시 실현
20          슬라이드 이동형 등 생활지원로봇 실증실험
21    1회용 비닐봉지 사용 금지 등 플라스틱 쓰레기 규제
22          자동차도로·집앞 보행로 제설규정 ‘손질’
23       접근·안전·저가성 기반 노인특화 교통전략 수립
24           옥외광고물 설치부터 관리까지 규제·지도
25     촘촘한 관리의 ‘광고물조례’ 별도 제정해 경관보호
26          활용도 낮은 가판대 10곳 새 용도 모색
27                          헤드라인뉴스
28                  